<a href="https://colab.research.google.com/github/shekhar-banerjee96/Quora_Question_Pairs/blob/main/Quora_Question_pairs_embeddings_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
!pip install kaggle

In [136]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory


In [137]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
from pathlib import Path

# Initialize Kaggle API
api = KaggleApi()
api.authenticate()

# Define dataset and download path
dataset = "quora/question-pairs-dataset"
os.makedirs('data',exist_ok=True)
download_path = Path("/content/data").as_posix()

# Check if the dataset already exists in the folder
dataset_files = os.listdir(download_path)


if any(dataset_files):
    print(f"Dataset folder has the follwing files already {dataset_files}")
else:
    # Download the dataset if it doesn't exist
    print(f"Downloading the dataset...")
    api.dataset_download_files(dataset, path=download_path, unzip=True)
    print(f"Dataset downloaded and extracted to {download_path}")

Dataset folder has the follwing files already ['questions.csv']


In [138]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import nltk , string

import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

stop_words = set(stopwords.words("english"))



import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [139]:
df = pd.read_csv('data/questions.csv')

In [140]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [141]:
df.shape

(404351, 6)

In [142]:
df.drop(['id'],axis=1,inplace=True)

In [143]:
df['is_duplicate'].value_counts(normalize=True)

,proportion
is_duplicate,
0,0.630752
1,0.369248


In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404351 entries, 0 to 404350
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   qid1          404351 non-null  int64 
 1   qid2          404351 non-null  int64 
 2   question1     404350 non-null  object
 3   question2     404349 non-null  object
 4   is_duplicate  404351 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 15.4+ MB


In [145]:
df.isnull().sum()

,0
qid1,0
qid2,0
question1,1
question2,2
is_duplicate,0


In [146]:
df.dropna(inplace=True)

In [147]:
df.isnull().sum()

,0
qid1,0
qid2,0
question1,0
question2,0
is_duplicate,0


In [148]:
qid = pd.Series(df['qid1'].tolist() + df['qid2'].tolist())
print('Number of unique questions : ',np.unique(qid).shape[0])
x = qid.duplicated()
print('Number of questions getting repeated : ',x.sum())

Number of unique questions :  789795
Number of questions getting repeated :  18901


In [149]:
!pip install clean-text

In [150]:
! pip install gensim

In [151]:
import gensim.downloader

print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [152]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

In [153]:
len(glove_vectors[0]) , len(glove_vectors)

(300, 400000)

In [154]:
type(glove_vectors)

gensim.models.keyedvectors.KeyedVectors

In [155]:
from cleantext import clean
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def oov_word2vec(word) :
  similar_word = glove_vectors.most_similar(word)
  return glove_vectors[similar_word]

def word2vectors(text):
    """
    Preprocesses the given text for NLP tasks: cleaning, tokenizing, and stopword removal.

    Args:
    text (str): Input text to preprocess.

    Returns:
    str: Preprocessed and cleaned text.
    """
    # Clean the text
    cleaned_text = clean(text,
                         clean_all = False,
                          extra_spaces = True,
                          stemming = False,
                          stopwords = True,
                          lowercase = True,
                          numbers = True,
                          punct = True,
                          stp_lang = 'english'
                         )
    lemmatizer = WordNetLemmatizer()


    cleaned_text = cleaned_text.split()
    cleaned_text = [lemmatizer.lemmatize(word) for word in cleaned_text]
    #print(f'Cleaned Text : {cleaned_text}')
    vector = [glove_vectors[i] for i in cleaned_text if i in glove_vectors   ]

        # Handle empty vectors: replace with zeros or another strategy
    if not vector:
        return np.zeros(glove_vectors.vector_size)  # Replace with zeros
    else:
        return np.mean(vector, axis=0)

In [156]:
text = "This is my new quikly"

vector = word2vectors(text)

print(vector)

[-6.2333e-01 -4.2434e-01 -3.5321e-02 -2.6693e-02  2.3120e-01  1.7631e-01
  2.8722e-01 -2.4921e-01  2.2212e-01 -1.6780e+00  3.7103e-01 -1.1762e-01
  7.7491e-02  9.0667e-02  3.9790e-01  6.2393e-01 -4.6144e-01 -9.5078e-02
  9.2384e-02 -1.4494e-02 -3.6806e-01  3.7096e-01  4.9309e-01  3.6888e-01
  1.0793e-01  1.7544e-01  2.5030e-01  4.4187e-01 -1.1506e-01  1.0503e-01
  2.0957e-01  1.9104e-01 -1.7090e-01  8.9148e-01 -1.0973e+00  4.5797e-01
  1.7506e-01 -1.4931e-04  3.5701e-01  1.5610e-01 -2.0892e-01  5.3822e-01
 -3.1083e-01  3.9404e-01  2.9395e-02  3.4538e-01 -2.3040e-02  3.6526e-01
 -9.1282e-02 -2.4338e-01 -9.7490e-02 -5.9859e-01 -1.5199e-01 -2.7588e-01
 -4.4608e-02  1.3343e-02 -1.7576e-01  6.6883e-02  1.2440e-01 -4.1805e-01
  9.1045e-02  1.0094e-01  2.7680e-01 -3.4494e-01  3.8171e-01  1.6173e-01
  3.3254e-01 -2.5571e-01  1.1421e-01  1.1263e-01 -6.4870e-02  1.8191e-01
 -4.0583e-02  4.4194e-03 -4.0280e-01  2.2875e-01 -4.8651e-01  2.0342e-01
 -1.3837e-01 -2.7807e-02 -4.0996e-01  1.7771e-01 -5

In [157]:
from tqdm import tqdm
tqdm.pandas()  # This initializes the progress bar for pandas apply

# Apply the preprocessing function with a progress bar
df['question1_vectors'] = df['question1'].progress_apply(word2vectors)

df['question2_vectors'] = df['question2'].progress_apply(word2vectors)

100%|██████████| 404348/404348 [03:34<00:00, 1887.07it/s]


In [158]:
df.head()

,qid1,qid2,question1,question2,is_duplicate,question1_vectors,question2_vectors
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[-0.04227572, -0.11327142, 0.13813183, -0.1614...","[-0.030426668, -0.036960002, 0.14941163, -0.12..."
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[0.21815749, 0.027459498, -0.16458973, -0.2364...","[0.07271722, 0.035750885, -0.18962076, 0.04213..."
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[-0.27179766, 0.14587234, 0.0655053, -0.135411...","[-0.09670981, 0.13422, 0.21341619, -0.03588559..."
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[0.008105333, 0.19356233, -0.122612, -0.052450...","[-0.22395867, 0.020949995, 0.120105326, -0.116..."
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[-0.15281779, 0.17652455, -0.04556456, -0.3691...","[0.045668, 0.103569984, -0.3742732, -0.287974,..."


In [159]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, Subtract, Multiply, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

from sklearn.model_selection import train_test_split


In [165]:
# Input dimensions for embeddings (e.g., 768 for SBERT)
embedding_dim = 300

# Input layers for the two sentence embeddings
input_1 = Input(shape=(embedding_dim,), name='Input1')
input_2 = Input(shape=(embedding_dim,), name='Input2')

# Shared dense layer for both inputs
shared_dense = Dense(128, activation='relu', name='SharedDense')

# Process each input through the shared dense layer
encoded_1 = shared_dense(input_1)
encoded_2 = shared_dense(input_2)

# Combine the two encoded inputs
# 1. Absolute difference
abs_diff = Lambda(lambda x: tf.abs(x[0] - x[1]), name='AbsDiff')([encoded_1, encoded_2])
# 2. Element-wise product
product = Multiply(name='ElementWiseProduct')([encoded_1, encoded_2])

# Concatenate features
combined = concatenate([abs_diff, product], name='CombineFeatures')

# Fully connected layers for classification
fc1 = Dense(64, activation='relu', name='FC1')(combined)
fc2 = Dense(32, activation='relu', name='FC2')(fc1)
output = Dense(1, activation='sigmoid', name='Output')(fc2)

# Define the model
siamese_model = Model(inputs=[input_1, input_2], outputs=output, name='SiameseModel')


In [166]:
siamese_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=BinaryCrossentropy(),
    metrics=['accuracy']
)
siamese_model.summary()

Model: "SiameseModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Input1 (InputLayer)       │ (None, 300)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Input2 (InputLayer)       │ (None, 300)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ SharedDense (Dense)       │ (None, 128)            │         38,528 │ Input1[0][0],          │
│                           │                        │                │ Input2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ AbsDiff (Lambda)          │ (None, 128)            │              0 │ SharedDense[0][0],     │
│                           │                        │                │ SharedDense[1][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ElementWiseProduct        │ (None, 128)            │              0 │ SharedDense[0][0],     │
│ (Multiply)                │                        │                │ SharedDense[1][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ CombineFeatures           │ (None, 256)            │              0 │ AbsDiff[0][0],         │
│ (Concatenate)             │                        │                │ ElementWiseProduct[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ FC1 (Dense)               │ (None, 64)             │         16,448 │ CombineFeatures[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ FC2 (Dense)               │ (None, 32)             │          2,080 │ FC1[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Output (Dense)            │ (None, 1)              │             33 │ FC2[0][0]              │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 57,089 (223.00 KB)

 Trainable params: 57,089 (223.00 KB)

 Non-trainable params: 0 (0.00 B)

In [162]:
type(df[['question1_vectors','question2_vectors']].values)

numpy.ndarray

In [163]:

X = df[['question1_vectors','question2_vectors']]
y = df['is_duplicate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Pad or truncate embeddings to ensure uniform length
def pad_embeddings(embeddings, max_length):
    """Pads or truncates embeddings to a fixed length."""
    padded_embeddings = []
    for embedding in embeddings:
        if len(embedding) < max_length:
            # Pad with zeros if shorter
            embedding = np.pad(embedding, (0, max_length - len(embedding)), 'constant')
        elif len(embedding) > max_length:
            # Truncate if longer
            embedding = embedding[:max_length]
        padded_embeddings.append(embedding)
    return np.array(padded_embeddings)

# Determine maximum embedding length
max_length = max(len(embedding) for embedding in X_train['question1_vectors'].values)
max_length2 = max(len(embedding) for embedding in X_train['question2_vectors'].values)
max_length = max(max_length,max_length2)
#print(f"Maximum embedding length: {max_length}")

# Pad or truncate embeddings in X_train
X1 = pad_embeddings(X_train['question1_vectors'].values, max_length)
X2 = pad_embeddings(X_train['question2_vectors'].values, max_length)


# Convert to tensors
X1 = tf.convert_to_tensor(X1, dtype=tf.float32)
X2 = tf.convert_to_tensor(X2, dtype=tf.float32)
y = y_train


siamese_model.fit(
    [X1, X2], y,
    batch_size=16,
    epochs=10,
    validation_split=0.3
)

